In [2]:
import torch
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer  
import pandas as pd
import re

In [3]:
corpus = torch.load('iu_xray_corpus.pt')

In [9]:
import pickle
with open('iu_xray_corpus.pickle', 'wb') as f:
    pickle.dump(corpus,f)

In [5]:
torch.save(corpus, 'iu_xray_corpus.pickle')

In [3]:
def clean_report_iu_xray(report):
    report_cleaner = lambda t: t.replace('..', '.').replace('..', '.').replace('..', '.').replace('1. ', '') \
        .replace('. 2. ', '. ').replace('. 3. ', '. ').replace('. 4. ', '. ').replace('. 5. ', '. ') \
        .replace(' 2. ', '. ').replace(' 3. ', '. ').replace(' 4. ', '. ').replace(' 5. ', '. ') \
        .strip().lower().split('. ')
    sent_cleaner = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').
                                    replace('\\', '').replace("'", '').strip().lower())
    tokens = [sent_cleaner(sent) for sent in report_cleaner(report) if sent_cleaner(sent) != []]
    report = ' . '.join(tokens) + ' .'
    return report

def clean_report_mimic_cxr(report):
    report_cleaner = lambda t: t.replace('\n', ' ').replace('__', '_').replace('__', '_').replace('__', '_') \
        .replace('__', '_').replace('__', '_').replace('__', '_').replace('__', '_').replace('  ', ' ') \
        .replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ') \
        .replace('..', '.').replace('..', '.').replace('..', '.').replace('..', '.').replace('..', '.') \
        .replace('..', '.').replace('..', '.').replace('..', '.').replace('1. ', '').replace('. 2. ', '. ') \
        .replace('. 3. ', '. ').replace('. 4. ', '. ').replace('. 5. ', '. ').replace(' 2. ', '. ') \
        .replace(' 3. ', '. ').replace(' 4. ', '. ').replace(' 5. ', '. ') \
        .strip().lower().split('. ')
    sent_cleaner = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '')
                                    .replace('\\', '').replace("'", '').strip().lower())
    tokens = [sent_cleaner(sent) for sent in report_cleaner(report) if sent_cleaner(sent) != []]
    report = ' . '.join(tokens) + ' .'
    return report

In [4]:
vectorizer = CountVectorizer(stop_words=None, min_df = 5e-4, tokenizer = lambda docs:clean_report_iu_xray(docs).split())
X = vectorizer.fit_transform(corpus)

In [5]:
tokens = vectorizer.get_feature_names_out()
all_tokens = torch.load('iu_xray_counter.pt')

In [6]:
for token in all_tokens:
    if token not in tokens:
        print(token)

definitive
one-third
circumscribed
syndesmophyte
empyema
hematoma
tb
midright
nondisplaced
satisfactory
findingsopacitiesinfiltrates
severely
evident
currently
reticular-nodular
orif
centered
air-filled
about
same
59
pannus
vasculaturity
compressive
telemetry
expansile
xxxxct
myeloma
tapering
b
testes
paraspinal
t7
demonstrable
atriumcavoatrial
elongation
20
posterobasal
benefit
followup
number
dominant
nodulemass
nondiagnostic
laterally
definitively
artifacts
submitted
tortuousectatic
appearances
interspaces
contrast-enhanced
suited
moderately-inflated
disruption
gastrostomy
7mm
origin
foci
questioned
limitsthe
retracted
superiorly
collapsed
stranding
pneumatoceles
additionally
pression
presence
morgagni
transmetatarsal
fracture-dislocations
erosions
infections
elbow
forearm
plantar
calcaneal
enthesophyte
diseaseemphysema
considerably
become
moved
outward
reaction
rods
positioned
subluxation
enteric
suction
arterial
generative
chin
cavitations
implantable
embolization
atelectasisairsp

In [7]:
X.shape

(2069, 966)

In [8]:
#获取词袋中所有文本关键词  
word = vectorizer.get_feature_names_out()  
#查看词频结果  
df_word =  pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())

# ---------------------------------------------------- 

#类调用  
transformer = TfidfTransformer(smooth_idf=True,norm='l2',use_idf=True)  
print(transformer)
#将计算好的词频矩阵X统计成TF-IDF值  
tfidf = transformer.fit_transform(X)  
#查看计算的tf-idf
df_word_tfidf = pd.DataFrame(tfidf.toarray(),columns=vectorizer.get_feature_names_out())
#查看计算的idf
df_word_idf = pd.DataFrame(list(zip(vectorizer.get_feature_names_out(),transformer.idf_)),columns=['单词','idf'])

TfidfTransformer()


/home/jun/anaconda3/envs/tc110_cd113/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
weights = tfidf.toarray()

In [10]:
weights.shape

(2069, 966)

In [11]:
df_word_tfidf.to_csv('iu_xray_train_tfidf.csv')